In [ ]:
from pathlib import Path

from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

from src.data_loader import load_f1_data
from src.features import engineer_features
from src.monte_carlo import MonteCarloSimulator
from src.regulation_transform import apply_2026_regulations

dataset = load_f1_data([2022, 2023, 2024, 2025])
features = engineer_features(dataset)
feature_cols = [c for c in features.columns if c not in {'position', 'driver_name', 'team_name', 'season', 'round', 'event_name'}]
X = features[feature_cols].fillna(features[feature_cols].mean())
y = features['position']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
model = XGBRegressor(n_estimators=100, max_depth=6, random_state=42).fit(X_train, y_train)
simulator = MonteCarloSimulator(model, feature_cols)
sample_race = features.groupby(['season', 'round']).nth(0).index[0]
season, rnd = sample_race
race_slice = features[(features['season'] == season) & (features['round'] == rnd)]
current = simulator.run(race_slice[feature_cols], race_slice['driver_name'])
future = simulator.run(apply_2026_regulations(race_slice[feature_cols]), race_slice['driver_name'])
{ 'current': current, 'future': future }